Make or Buy Problem

Install Optimization package

In [3]:
pip install PulP

Note: you may need to restart the kernel to use updated packages.


In [4]:
from pulp import * 
import pulp as p

Input values

In [5]:
#Create a list of lawn trimmers
Models = ['Electric', 'Gas']

# Create a dictionary for the unit cost to make
unit_make = {'Electric':55, 'Gas':85}

# Create a dictionary for the unit cost to buy
unit_buy = {'Electric':67, 'Gas':95}

# Create a dictionary for Production
production = {'Electric':0.20, 'Gas':0.40}

# Create a dictionary for Assembly
assembly = {'Electric':0.30, 'Gas':0.50}

# Create a dictionary for Packaging
packaging = {'Electric':0.10, 'Gas':0.10}

# Create a dictionary for the requested quantity
number_ordered = {'Electric':30000, 'Gas':15000}

# Define a parameter for the available labor production
Ava_production = 10000

# Define a parameter for the available labor assembly
Ava_assembly = 15000

# Define a parameter for the available labor packaging
Ava_packaging = 5000

Define decision variables

In [6]:
Make_vars = LpVariable.dicts("Make", Models, lowBound = 0, upBound = None, cat = LpContinuous)
Buy_vars = LpVariable.dicts("Buy", Models, lowBound = 0, upBound = None, cat = LpContinuous)

Create a variable to hold problem data

In [7]:
Make_Buy = p.LpProblem("Weedwacker_company", p.LpMinimize)

Define objective function

In [8]:
Make_Buy += lpSum(Make_vars[i]*unit_make[i] for i in Models) + lpSum(Buy_vars[i]*unit_buy[i] for i in Models) 

Define the constraints

In [9]:
# Production Constraint
Make_Buy += lpSum(production[model]*Make_vars[model] for model in Models) <= Ava_production

# Assembly Constraint
Make_Buy += lpSum(assembly[model]*Make_vars[model] for model in Models) <= Ava_assembly

# Assembly Constraint
Make_Buy += lpSum(packaging[model]*Make_vars[model] for model in Models) <= Ava_packaging

# Demand constraint
for model in Models:
    Make_Buy += Make_vars[model] + Buy_vars[model] == number_ordered[model]

In [10]:
Make_Buy

Weedwacker_company:
MINIMIZE
67*Buy_Electric + 95*Buy_Gas + 55*Make_Electric + 85*Make_Gas + 0
SUBJECT TO
_C1: 0.2 Make_Electric + 0.4 Make_Gas <= 10000

_C2: 0.3 Make_Electric + 0.5 Make_Gas <= 15000

_C3: 0.1 Make_Electric + 0.1 Make_Gas <= 5000

_C4: Buy_Electric + Make_Electric = 30000

_C5: Buy_Gas + Make_Gas = 15000

VARIABLES
Buy_Electric Continuous
Buy_Gas Continuous
Make_Electric Continuous
Make_Gas Continuous

In [11]:
status = Make_Buy.solve()
print(p.LpStatus[status])

Optimal


In [12]:
print("Total cost = $", p.value(Make_Buy.objective))

Total cost = $ 2975000.0


In [13]:
for variable in Make_Buy.variables():
    print(variable.name, "=", variable.varValue)

Buy_Electric = 0.0
Buy_Gas = 5000.0
Make_Electric = 30000.0
Make_Gas = 10000.0
